In [13]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [ ]:
parcel = 1

# Prepare data
# Import dataset
df = pd.read_csv('data/parcel_{}_combined.csv'.format(parcel), delimiter=',')
df.head()

# Convert fecha column to datetime
df['date'] = pd.to_datetime(df['date'])

# Make date as index
df = df.set_index('date')

num_features = df.shape[1]

# interpolate missing values
df = df.interpolate(method='linear', axis=0)

# remove all nan values
df = df.dropna()

# Get the minimum and maximum values of 'eto'
min_et = df['eto'].min()
max_et = df['eto'].max()

# Define the list of columns to consider for the grid search, excluding 'eto'
columns_to_search = [column for column in df.columns if column != 'eto']

# Define the range of column counts to consider (e.g., from 1 to the total number of columns - 1)
min_columns = 0
max_columns = len(columns_to_search)

# Define a variable to keep track of the best model and its performance
best_model = None
best_mse = float('inf')

# Perform grid search by iterating through different combinations of columns
for num_columns in range(min_columns, max_columns + 1):
    print("Number of columns: {}".format(num_columns))

    # Generate all possible combinations of columns for the current number of columns
    column_combinations = itertools.combinations(columns_to_search, num_columns)

    # Iterate through each combination of columns
    for columns in column_combinations:

        # Prepare data using the selected columns (including 'Evapotranspiration')
        selected_columns = list(columns) + ['eto']
        print("Selected columns: {}".format(selected_columns))

        # Normalize data
        scaled_data = df[selected_columns].values

        scalers = {}
        for column in df[selected_columns].columns:
            scalers[column] = MinMaxScaler(feature_range=(0, 1))
            scaled_data[:, df[selected_columns].columns.get_loc(column)] = scalers[column].fit_transform(scaled_data[:, df[selected_columns].columns.get_loc(column)].reshape(-1, 1)).flatten()

        # Prepare input and output sequences
        n_days = 7  # Number of days in each sequence
        X = []
        y = []
        for i in range(n_days, len(df[selected_columns]) - n_days + 1):
            X.append(scaled_data[i-n_days:i])
            y.append(scaled_data[i:i+n_days, df[selected_columns].columns.get_loc('eto')])  # Output sequence with 7 days of Evapotranspiration


        X = np.array(X)
        y = np.array(y)

        # Define the callbacks
        early_stopping = EarlyStopping(patience=10, monitor='val_loss', mode='min', restore_best_weights=True)
        # model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
        lr_scheduler = LearningRateScheduler(lambda epoch: 0.001 * 0.9 ** epoch)  # Example learning rate scheduler

        # Define LSTM model
        model = keras.Sequential()
        model.add(keras.layers.LSTM(64, activation='relu', input_shape=(n_days, X.shape[2])))
        model.add(keras.layers.Dense(n_days))
        model.compile(optimizer='adam', loss='mse')


        # Apply TimeSeriesSplit for cross-validation
        mse_scores = []
        mae_scores = []
        tscv = TimeSeriesSplit(n_splits=5, test_size=1)  # Assuming you want to split the data into 5 folds
        for fold, (train_index, test_index) in enumerate(tscv.split(X), 1):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Train the model
            model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping, lr_scheduler], verbose=0)

            # Evaluate the model
            loss = model.evaluate(X_test, y_test, verbose=0)
            # print(f"Fold {fold} - Test Loss: {loss}")

            # Make predictions
            predictions = model.predict(X_test, verbose=0)

            # Inverse transform the predictions and actual values
            inv_predictions = scalers['eto'].inverse_transform(predictions).flatten()
            inv_actual = scalers['eto'].inverse_transform(y_test).flatten()
            inv_input = scalers['eto'].inverse_transform(X_test[:, :, df[selected_columns].columns.get_loc('eto')].reshape(1, -1)).flatten()

            # Calculate MSE
            mse = mean_squared_error(inv_actual, inv_predictions)
            mse_scores.append(mse)
            # print(f"Fold {fold} - MSE: {mse}")

            # Calculate MAE
            mae = np.mean(np.abs(inv_actual - inv_predictions))
            mae_scores.append(mae)
            # print(f"Fold {fold} - MAE: {mae}")

        # Calculate average MSE across all splits
        print(f"Average MSE: {np.mean(mse_scores)}")
        print(f"Average MAE: {np.mean(mae_scores)}")
        print()

        # Update the best model and MSE if the current model performs better
        if np.mean(mse_scores) < best_mse:
            best_columns = selected_columns
            best_model = model
            best_mse = np.mean(mse_scores)

# Print the best model and MSE
print(f"Best columns: {best_columns}")
print(f"Best MSE: {best_mse}")
